# Prediction of sales

### Problem Statement
[The dataset](https://drive.google.com/file/d/1B07fvYosBNdIwlZxSmxDfeAf9KaygX89/view?usp=sharing) represents sales data for 1559 products across 10 stores in different cities. Also, certain attributes of each product and store are available. The aim is to build a predictive model and find out the sales of each product at a particular store.

|Variable|Description|
|: ------------- |:-------------|
|Item_Identifier|Unique product ID|
|Item_Weight|Weight of product|
|Item_Fat_Content|Whether the product is low fat or not|
|Item_Visibility|The % of total display area of all products in a store allocated to the particular product|
|Item_Type|The category to which the product belongs|
|Item_MRP|Maximum Retail Price (list price) of the product|
|Outlet_Identifier|Unique store ID|
|Outlet_Establishment_Year|The year in which store was established|
|Outlet_Size|The size of the store in terms of ground area covered|
|Outlet_Location_Type|The type of city in which the store is located|
|Outlet_Type|Whether the outlet is just a grocery store or some sort of supermarket|
|Item_Outlet_Sales|Sales of the product in the particulat store. This is the outcome variable to be predicted.|

Please note that the data may have missing values as some stores might not report all the data due to technical glitches. Hence, it will be required to treat them accordingly.



### Explore the problem in following stages:

1. Hypothesis Generation – understanding the problem better by brainstorming possible factors that can impact the outcome
2. Data Exploration – looking at categorical and continuous feature summaries and making inferences about the data.
3. Data Cleaning – imputing missing values in the data and checking for outliers
4. Feature Engineering – modifying existing variables and creating new ones for analysis
5. Model Building – making predictive models on the data

In [3]:
import pandas as pd
import numpy as np

#Read files:
data = pd.read_csv("regression_exercise.csv", delimiter=',')
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [4]:
data['Item_Weight'] = data['Item_Weight'].fillna(12.8)
data['Outlet_Size'] = data['Outlet_Size'].fillna('empty')
data.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

## 4. Feature Engineering

1. Resolving the issues in the data to make it ready for the analysis.
2. Create some new variables using the existing ones.





### Create a broad category of Type of Item

Item_Type variable has many categories which might prove to be very useful in analysis. Look at the Item_Identifier, i.e. the unique ID of each item, it starts with either FD, DR or NC. If you see the categories, these look like being Food, Drinks and Non-Consumables. Use the Item_Identifier variable to create a new column

In [5]:

data['Item_Category'] = ''
my_words = {'FD': 'Food', 'DR': 'Drinks', 'NC': 'Non-Consumables'}
for word in my_words.keys():
    data.loc[data['Item_Identifier'].str.startswith(word, na=False), ['Item_Category']] = my_words[word]

In [6]:
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Category
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,Food
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,Drinks
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,Food
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,empty,Tier 3,Grocery Store,732.3800,Food
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,Non-Consumables


### Determine the years of operation of a store

Make a new column depicting the years of operation of a store (i.e. how long the store exists). 

In [7]:
current_year = 2021
data['Years_of_Operation'] = current_year - data['Outlet_Establishment_Year']

### Modify categories of Item_Fat_Content

There are difference in representation in categories of Item_Fat_Content variable. This should be corrected.

In [8]:
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Category,Years_of_Operation
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,Food,22
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,Drinks,12
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,Food,22
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,empty,Tier 3,Grocery Store,732.3800,Food,23
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,Non-Consumables,34


In [9]:
data['Modified_Fat_Content'] = ''
my_words = {'LF': 'Low Fat', 'Low Fat': 'Low Fat', 'low fat': 'Low Fat', 'Regular': 'Regular', 'reg': 'Regular'}
for word in my_words.keys():
    data.loc[data['Item_Fat_Content'].str.contains(word, na=False), ['Modified_Fat_Content']] = my_words[word]

There are some non-consumables as well and a fat-content should not be specified for them. Create a separate category for such kind of observations.

In [10]:
my_words = {'NC': 'No Fat NC'}
for word in my_words.keys():
    data.loc[data['Item_Identifier'].str.startswith(word, na=False), ['Modified_Fat_Content']] = my_words[word]

### Numerical and One-Hot Encoding of Categorical variables

Since scikit-learn algorithms accept only numerical variables, we need to convert all categorical variables into numeric types. 

- if the variable is Ordinal we can simply map its values into numbers
- if the variable is Nominal (we cannot sort the values) we need to One-Hot Encode them --> create dummy variables

In [11]:
data.groupby('Outlet_Size').count()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Category,Years_of_Operation,Modified_Fat_Content
Outlet_Size,,,,,,,,,,,,,,
High,932,932,932,932,932,932,932,932,932,932,932,932,932,932
Medium,2793,2793,2793,2793,2793,2793,2793,2793,2793,2793,2793,2793,2793,2793
Small,2388,2388,2388,2388,2388,2388,2388,2388,2388,2388,2388,2388,2388,2388
empty,2410,2410,2410,2410,2410,2410,2410,2410,2410,2410,2410,2410,2410,2410


In [72]:
data['Is_Food'] = ''
food = {'Food': 1}
for word in food.keys():
    data.loc[data['Item_Category'].str.contains(word, na=False), ['Is_Food']] = food[word]
    
data['Is_Food'] = data['Is_Food'].replace('', 0)

In [73]:
data['Is_Drink'] = ''
drinks = {'Drinks': 1}
for word in drinks.keys():
    data.loc[data['Item_Category'].str.contains(word, na=False), ['Is_Drink']] = drinks[word]
    
data['Is_Drink'] = data['Is_Drink'].replace('', 0)

In [75]:
data['Is_LowFat'] = ''
low_fat = {'Low Fat': 1}
for word in low_fat.keys():
    data.loc[data['Modified_Fat_Content'].str.contains(word, na=False), ['Is_LowFat']] = low_fat[word]
    
data['Is_LowFat'] = data['Is_LowFat'].replace('', 0)

In [77]:
data['Is_RegFat'] = ''
reg_fat = {'Regular': 1}
for word in reg_fat.keys():
    data.loc[data['Modified_Fat_Content'].str.contains(word, na=False), ['Is_RegFat']] = reg_fat[word]
    
data['Is_RegFat'] = data['Is_RegFat'].replace('', 0)

In [85]:
data['Is_SupType1'] = ''
sup_type1 = {'Supermarket Type1': 1}
for word in sup_type1.keys():
    data.loc[data['Outlet_Type'].str.contains(word, na=False), ['Is_SupType1']] = sup_type1[word]
    
data['Is_SupType1'] = data['Is_SupType1'].replace('', 0)

In [87]:
data['Is_SupType2'] = ''
sup_type2 = {'Supermarket Type2': 1}
for word in sup_type2.keys():
    data.loc[data['Outlet_Type'].str.contains(word, na=False), ['Is_SupType2']] = sup_type2[word]
    
data['Is_SupType2'] = data['Is_SupType2'].replace('', 0)

In [88]:
data['Is_SupType3'] = ''
sup_type3 = {'Supermarket Type3': 1}
for word in sup_type3.keys():
    data.loc[data['Outlet_Type'].str.contains(word, na=False), ['Is_SupType3']] = sup_type3[word]
    
data['Is_SupType3'] = data['Is_SupType3'].replace('', 0)

In [90]:
data['Is_Tier1'] = ''
loc_tier1 = {'Tier 1': 1}
for word in loc_tier1.keys():
    data.loc[data['Outlet_Location_Type'].str.contains(word, na=False), ['Is_Tier1']] = loc_tier1[word]
    
data['Is_Tier1'] = data['Is_Tier1'].replace('', 0)

In [91]:
data['Is_Tier2'] = ''
loc_tier2 = {'Tier 2': 1}
for word in loc_tier2.keys():
    data.loc[data['Outlet_Location_Type'].str.contains(word, na=False), ['Is_Tier2']] = loc_tier2[word]
    
data['Is_Tier2'] = data['Is_Tier2'].replace('', 0)

In [94]:
data['Is_OutletSmall'] = ''
outlet_small = {'Small': 1}
for word in outlet_small.keys():
    data.loc[data['Outlet_Size'].str.contains(word, na=False), ['Is_OutletSmall']] = outlet_small[word]
    
data['Is_OutletSmall'] = data['Is_OutletSmall'].replace('', 0)

In [95]:
data['Is_OutletMedium'] = ''
outlet_medium = {'Medium': 1}
for word in outlet_medium.keys():
    data.loc[data['Outlet_Size'].str.contains(word, na=False), ['Is_OutletMedium']] = outlet_medium[word]
    
data['Is_OutletMedium'] = data['Is_OutletMedium'].replace('', 0)

In [96]:
data['Is_OutletHigh'] = ''
outlet_high = {'High': 1}
for word in outlet_high.keys():
    data.loc[data['Outlet_Size'].str.contains(word, na=False), ['Is_OutletHigh']] = outlet_high[word]
    
data['Is_OutletHigh'] = data['Is_OutletHigh'].replace('', 0)

All variables should be by now numeric

### Exporting Data
you can save the processed data to disk

In [99]:
data.to_csv(r'\Home\Hafsa\my_project\data_featEng.csv')